In [1]:
import tensorflow as tf

def pipe(data, batch_size = 128, shuffle = False):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    if shuffle:
        dataset = dataset.shuffle(buffer_size = batch_size * 10)
    dataset = dataset.batch(batch_size)
    #dataset = dataset.prefetch((batch_size * 2) + 1)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

(tr_x, tr_y), (te_x, te_y) = tf.keras.datasets.cifar10.load_data()

tr_x = tr_x * 1/255
te_x = te_x * 1/255

batch_size = 128

tr_data = pipe((tr_x, tr_y), batch_size = batch_size, shuffle = True)
te_data = pipe((te_x, te_y), batch_size = batch_size, shuffle = False)

170500096/170498071 [==============================] - 4s 0us/step


In [2]:
import resnest

model = resnest.resnest101(input_shape = (32, 32, 3), include_top = False, weights = "imagenet", input_tensor = None, classes = 1000)

flatten = tf.keras.layers.GlobalAveragePooling2D()(model.output)
drop_out = tf.keras.layers.Dropout(0.5)(flatten)
dense = tf.keras.layers.Dense(2048, activation = "relu")(drop_out)
prediction = tf.keras.layers.Dense(10, activation = "softmax", name = "prediction")(dense)
model = tf.keras.Model(model.input, prediction)

Downloading: "https://s3.us-west-1.wasabisys.com/resnest/torch/resnest101-22405ba7.pth" to /root/.cache/torch/hub/checkpoints/resnest101-22405ba7.pth


In [3]:
loss = tf.keras.losses.sparse_categorical_crossentropy
opt = tf.keras.optimizers.Adam(1e-4)
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, optimizer = opt, metrics = metric)

In [4]:
model.fit(tr_data, validation_data = te_data, epochs = 50)

Epoch 1/50
391/391 [==============================] - 56s 144ms/step - loss: 2.2179 - sparse_categorical_accuracy: 0.2226 - val_loss: 2.2187 - val_sparse_categorical_accuracy: 0.1797
Epoch 2/50
391/391 [==============================] - 53s 135ms/step - loss: 1.5344 - sparse_categorical_accuracy: 0.4758 - val_loss: 1.1841 - val_sparse_categorical_accuracy: 0.5939
Epoch 3/50
391/391 [==============================] - 52s 134ms/step - loss: 0.9328 - sparse_categorical_accuracy: 0.6882 - val_loss: 1.1895 - val_sparse_categorical_accuracy: 0.7486
Epoch 4/50
391/391 [==============================] - 52s 133ms/step - loss: 0.6582 - sparse_categorical_accuracy: 0.7799 - val_loss: 1.6012 - val_sparse_categorical_accuracy: 0.7847
Epoch 5/50
391/391 [==============================] - 52s 133ms/step - loss: 0.4902 - sparse_categorical_accuracy: 0.8364 - val_loss: 2.7330 - val_sparse_categorical_accuracy: 0.8052
Epoch 6/50
391/391 [==============================] - 52s 133ms/step - loss: 0.3808 -

In [5]:
with open("model.json", mode = "w") as file:
    file.write(model.to_json())
model.save_weights("model.h5")

In [6]:
with open("model.json", mode = "r") as file:
    model = tf.keras.models.model_from_json(file.read(), {"rSoftMax":resnest.rSoftMax})
model.load_weights("model.h5")

In [7]:
loss = tf.keras.losses.sparse_categorical_crossentropy
metric = [tf.keras.metrics.sparse_categorical_accuracy]
model.compile(loss = loss, metrics = metric)
model.evaluate(te_data)

79/79 [==============================] - 3s 37ms/step - loss: 0.6120 - sparse_categorical_accuracy: 0.8650


[0.6120349168777466, 0.8650000095367432]